# Data Cleaning

# Imports

In [1]:
import pandas as pd
import datetime

# Load Data

In [3]:
# Read csv
df = pd.read_csv("Data/frmgham2.csv")
df.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0,52,121.0,66.0,0,0.0,NaN,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0,52,105.0,69.5,0,0.0,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [6]:
df.columns

Index(['RANDID', 'SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE',
       'CIGPDAY', 'BMI', 'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'educ',
       'PREVCHD', 'PREVAP', 'PREVMI', 'PREVSTRK', 'PREVHYP', 'TIME', 'PERIOD',
       'HDLC', 'LDLC', 'DEATH', 'ANGINA', 'HOSPMI', 'MI_FCHD', 'ANYCHD',
       'STROKE', 'CVD', 'HYPERTEN', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD',
       'TIMESTRK', 'TIMECVD', 'TIMEDTH', 'TIMEHYP'],
      dtype='object')

# Clean Column 

In [7]:
# label the appropriate sex
sex_dict = {1:"Male", 2:"Female"}
df['SEX'] = [sex_dict[key] for key in df['SEX']]
# label the period
period_dict = {1:"One", 2:"Two", 3:"Three"}
df['PERIOD'] = [period_dict[key] for key in df['PERIOD']]

df.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,Male,195.0,39,106.0,70.0,0,0.0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,Male,209.0,52,121.0,66.0,0,0.0,NaN,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,Female,250.0,46,121.0,81.0,0,0.0,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,Female,260.0,52,105.0,69.5,0,0.0,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,Female,237.0,58,108.0,66.0,0,0.0,28.50,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [8]:
df.count()

RANDID      11627
SEX         11627
TOTCHOL     11218
AGE         11627
SYSBP       11627
DIABP       11627
CURSMOKE    11627
CIGPDAY     11548
BMI         11575
DIABETES    11627
BPMEDS      11034
HEARTRTE    11621
GLUCOSE     10187
educ        11332
PREVCHD     11627
PREVAP      11627
PREVMI      11627
PREVSTRK    11627
PREVHYP     11627
TIME        11627
PERIOD      11627
HDLC         3027
LDLC         3026
DEATH       11627
ANGINA      11627
HOSPMI      11627
MI_FCHD     11627
ANYCHD      11627
STROKE      11627
CVD         11627
HYPERTEN    11627
TIMEAP      11627
TIMEMI      11627
TIMEMIFC    11627
TIMECHD     11627
TIMESTRK    11627
TIMECVD     11627
TIMEDTH     11627
TIMEHYP     11627
dtype: int64

In [11]:
# Subset only ten years
tenYears = (365*8) + (366*2) # Two leap years in ten years
subj_exclude = list(df[((df.PREVSTRK==1)&(df.PERIOD == 'One'))].RANDID) # Exclude subjects who had PREVSTRK at Period 1
df = df[~df.RANDID.isin(subj_exclude)]

df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            11617, 11618, 11619, 11620, 11621, 11622, 11623, 11624, 11625,
            11626],
           dtype='int64', length=11567)

In [12]:
# Restructure the data to have baseline risk
df_wide = df.pivot(index = "RANDID", columns="PERIOD")
df_wide.columns = ['_'.join(col).strip() for col in df_wide.columns.values]
df_wide = df_wide.reset_index()
df_wide["STROKE"] = [x if b <= tenYears and b > 0 else 0 for b, x in zip(df_wide["TIMESTRK_One"], df_wide["STROKE_One"])]
df_wide["TIME"] = [x if x <= tenYears else tenYears for x in df_wide["TIMESTRK_One"]]

In [13]:
df_wide.head()

,RANDID,SEX_One,SEX_Three,SEX_Two,TOTCHOL_One,TOTCHOL_Three,TOTCHOL_Two,AGE_One,AGE_Three,AGE_Two,...,TIMECVD_Three,TIMECVD_Two,TIMEDTH_One,TIMEDTH_Three,TIMEDTH_Two,TIMEHYP_One,TIMEHYP_Three,TIMEHYP_Two,STROKE,TIME
0,2448,Male,Male,NaN,195.0,209.0,NaN,39.0,52.0,NaN,...,6438.0,NaN,8766.0,8766.0,NaN,8766.0,8766.0,NaN,0.0,3652.0
1,6238,Female,Female,Female,250.0,237.0,260.0,46.0,58.0,52.0,...,8766.0,8766.0,8766.0,8766.0,8766.0,8766.0,8766.0,8766.0,0.0,3652.0
2,9428,Male,NaN,Male,245.0,NaN,283.0,48.0,NaN,54.0,...,NaN,8766.0,8766.0,NaN,8766.0,8766.0,NaN,8766.0,0.0,3652.0
3,10552,Female,NaN,Female,225.0,NaN,232.0,61.0,NaN,67.0,...,NaN,2089.0,2956.0,NaN,2956.0,0.0,NaN,0.0,1.0,2089.0
4,11252,Female,Female,Female,285.0,NaN,343.0,46.0,58.0,51.0,...,8766.0,8766.0,8766.0,8766.0,8766.0,4285.0,4285.0,4285.0,0.0,3652.0


In [14]:
# Subset columns that we are interested in using
riskFactors = ['AGE_One',
               'PREVCHD_One',
               'BMI_One',
               'BPMEDS_One',
               'SYSBP_One',
               'CURSMOKE_One',
               'TOTCHOL_One',
               'DIABETES_One',
               'SEX_One',
               'TIME',
               'STROKE']
nms = ['Age',
       'Prevalent CHD',
       'BMI',
       'Anti-Hypertensives',
       'Systolic blood pressure',
       'Smoke',
       'Cholesterol',
       'Diabetes',
       'Sex',
       'Time',
       'Stroke']
df_subset = df_wide[riskFactors]
df_subset.reset_index(inplace = True, drop = True)
df_subset.columns = nms

In [15]:
df_subset.head()

,Age,Prevalent CHD,BMI,Anti-Hypertensives,Systolic blood pressure,Smoke,Cholesterol,Diabetes,Sex,Time,Stroke
0,39.0,0.0,26.97,0.0,106.0,0.0,195.0,0.0,Male,3652.0,0.0
1,46.0,0.0,28.73,0.0,121.0,0.0,250.0,0.0,Female,3652.0,0.0
2,48.0,0.0,25.34,0.0,127.5,1.0,245.0,0.0,Male,3652.0,0.0
3,61.0,0.0,28.58,0.0,150.0,1.0,225.0,0.0,Female,2089.0,1.0
4,46.0,0.0,23.10,0.0,130.0,1.0,285.0,0.0,Female,3652.0,0.0


In [17]:
# Save the wide format subset cleaned data
df_subset.to_csv("Data/cleaned_wideData.csv", index=False)
# Save the cleaned long format data
df.to_csv("Data/cleaned_data.csv", index = False)